In [1]:
import sys
import matplotlib.pyplot as plt

In [16]:
class Trader():
    def __init__(self):
        self.fund = 10000 # money in pocket
        self.bought = 0 # price when bought
        self.share = 0 # shares holding
    def buy(self, data, day):
        if(not self.bought):
            self.bought = data[day]
            self.share = self.fund//self.bought
            self.fund = self.fund%self.bought
    def sell(self, data, day):
        if(self.bought > 0):
            self.fund += self.share * data[day]
            self.bought = 0
            self.share = 0
            #print('sell on day {}: {}'.format(day, self.fund))
    def final(self, data):
        return self.fund + self.share*data[-1]
    def reset(self):
        self.fund = 10000 # money in pocket
        self.bought = 0 # price when bought
        self.share = 0 # shares holding

In [17]:
class Strat_Naive(Trader):
    # set argment number and limitation
    name = 'naive'
    arg_count = 0
    # Trade with the strategy
    def __call__(self, data, *args):
        pre_price  = 0
        for day in range(len(data)):
            if data[day] > pre_price:
                self.buy(data, day)
            else:
                self.sell(data, day)
            pre_price = data[day]
        return self.final(data)

In [31]:
class Strat_Conserv_Naive(Trader):
    # set argment number and limitation
    name = 'conserv'
    arg_count = 1
    # Trade with the strategy
    def __call__(self, data, *args):
        arg1 = args[0]//20
        self.buy(data, 0)
        rise4 = 0
        fall4 = 0
        
        pre_price  = 0
        for day in range(len(data)):
            if data[day] > pre_price:
                rise4+=1
                if rise4 > arg1:
                    rise4 = 0
                    self.buy(data, day)
            else:
                fall4+=1
                if fall4 > arg1:
                    fall4 = 0
                    self.sell(data, day)
            pre_price = data[day]
        #print(self.final(data), arg1)
        return self.final(data)

In [32]:
class Strat_Lazy(Trader):
    name = 'lazy'
    arg_count = 0
    argm = []
    argM = []
    def __call__(self, data, *args):
        self.buy(data, 0)
        self.sell(data, len(data) - 1)
        return self.final(data)

In [33]:
class Evaluate:
    pw = [100000000, 1000000, 10000, 100]
    def __init__(self, cls, data):
        self.cls = cls
        self.data = data
    def FindOptArgs(self):
        maxVal = 0
        maxArg = 0
        for i in range(100**self.cls.arg_count):
            #args = []
            #print(args)
            tmp = self.cls(self.data, i%100, i/self.pw[3]%100, i/self.pw[2]%100, i/self.pw[1]%100, i/self.pw[0])
            #print('###', tmp, i)
            if  tmp > maxVal:
                maxVal = tmp
                maxArg = i
            self.cls.reset()
        return maxVal, maxArg
    def ReturnRate(self, *args):
        return self.cls(self.data, *args)
    def name(self):
        return self.cls.name

In [36]:
# Process input
#DSI, IAU, LQD, SPY
files = ['DSI.csv', 'IAU.csv', 'LQD.csv', 'SPY.csv']
for file in files:
    f = open(file, 'r')
    closeP = []
    i = 0
    f.readline()
    while(True):
        line = f.readline()
        if not line:
            break
        closeP.append(float(line.split(',')[5]))
    s = Strat_Naive()
    e = Evaluate(s, closeP)
    print(e.name(), e.FindOptArgs(), file)
    #plt.plot(closeP, label=file)
    #plt.show()

naive (10617.971079000026, 0) DSI.csv
naive (10828.791561999973, 0) IAU.csv
naive (12946.338492000024, 0) LQD.csv
naive (9925.184277000011, 0) SPY.csv


In [25]:
print()